In [1]:
from matplotlib import pyplot as plt
from utils.utils import *
import utils.promethee_functions as pf

utils.py Loading


# K-Medoids using Promethee Gamma as distance measure

In [3]:
data = read_data()

column = data["co2prod"]
score = pf.score_function(column, maximize=False)

data["co2prod"] = score

# Remove the last 3 rows
data = data.iloc[:-3]
data.tail(2)


,iso3,co2prod,hdi,le,gdi,eys,mys
187,URY,"[32.132821846999995, 31.961541150999995, 31.77...","[0.702, 0.706, 0.709, 0.712, 0.718, 0.72, 0.72...","[73.162, 73.305, 73.441, 73.579, 73.728, 73.89...","[1.037, 1.033, 1.033, 1.029, 1.024, 1.02, 1.01...","[12.9682703, 13.05552959, 12.86606026, 12.9869...","[7.151282631, 7.189396058, 7.227509486, 7.2656..."
193,ZMB,"[33.092447830999994, 33.103370289999994, 33.10...","[0.417, 0.411, 0.408, 0.412, 0.407, 0.407, 0.4...","[47.926, 47.097, 46.512, 46.209, 45.854, 45.55...","[0.78, 0.818, 0.827, 0.829, 0.829, 0.833, 0.83...","[7.79542017, 7.854483475, 7.91354678, 7.972610...","[4.481160164, 4.572166937, 4.66317371, 4.75418..."


In [5]:
def get_min_max_criteria(data):
    min_max = {}
    for column in data.columns[1:]:
        min_max[column] = {"min": float('inf'), "max": float('-inf')}
        # Each column is a column of time series data
        column_data = data[column]

        for row in column_data:
            tempmin = row.min()
            tempmax = row.max()
            if tempmin < min_max[column]["min"]:
                min_max[column]["min"] = tempmin
            if tempmax > min_max[column]["max"]:
                min_max[column]["max"] = tempmax

        # Print the min and max values for each criteria
    for key, value in min_max.items():
        print(f"{key}: min={round(value.get('min'),4)}, max={round(value.get('max'),4)}")


get_min_max_criteria(data)

co2prod: min=0.0, max=33.3863
hdi: min=0.257, max=0.967
le: min=37.105, max=85.473
gdi: min=0.383, max=1.041
eys: min=3.5751, max=23.2477
mys: min=1.4606, max=14.2559


In [6]:
L = data.iloc[0]["co2prod"].shape[0] # Length of the time series
N = data.shape[0] # Number of time series
K = data.columns.shape[0] -1 # Number of features/criteria

W = [1/K for _ in range(K)]
P = [0.90 for _ in range(K)]
Q = [0.1 for _ in range(K)]
criterias = data.columns[1:]
alternatives = data["iso3"].values

phi_c_all = pf.get_all_Phi_c(data, P, Q)
PHI = pf.PHI_all(phi_c_all, W, N, L, K)

c:\Users\raphu\Documents\ULB\MA2\ulb-master-thesis\code\src\utils\promethee_functions.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  d = a_i[c] - a_j[c]


The idea consist of using the $\gamma_{ij}$ values as a distance measure for the K-Means algorithm
- Build $\eta_{ij}=\gamma_{ij}+\gamma_{ji}$ (for each time stemp)
- Aggregate the $\eta_{ij}$ series into one value that will be used as distance between two alternatives

In [7]:
def get_eta_matrix(data, phi_c_all, W):
    N = data.shape[0]
    L = data.iloc[0]["co2prod"].shape[0]
    gamma = pf.get_gamma_matrix(data, phi_c_all, W)
    eta = np.zeros((N, N, L))
    for i in range(N):
        for j in range(N):
            for l in range(L):
                eta[i, j, l] = gamma[i, j, l] + gamma[j, i, l]
    return eta

def aggregate_series(series, weight_vector):
    """
        Aggregate a time series to a single value using a time weight vector (one weight per time point)
    """
    return np.dot(series, weight_vector)

def aggregate_all_series(data, weight_vector):
    """ 
        Aggregate all time series of a matrix NxNxL using a weight vector (one weight per time point)
    """
    # Nb of alternatives = data.shape[0]
    # Nb of time points = data.shape[2]
    N = data.shape[0]
    L = data.shape[2]
    aggregated_series = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            aggregated_series[i, j] = aggregate_series(data[i, j, :], weight_vector)

    return aggregated_series


weight_vector = [1/L for _ in range(L)] # Equal weights for all time points


eta = get_eta_matrix(data, phi_c_all, W)
agg_eta = aggregate_all_series(eta, weight_vector)

# Transform the agg_eta into a dataframe with the iso3 codes as index and columns

agg_eta_df = pd.DataFrame(agg_eta, index=alternatives, columns=alternatives)
agg_eta_df

,ALB,ARG,AUS,AUT,BHS,BHR,BGD,BRB,BEL,BOL,...,SWE,CHE,SYR,THA,TTO,TUR,GBR,USA,URY,ZMB
ALB,0.000000,0.279711,0.719044,0.457924,0.374087,0.375153,0.431331,0.255879,0.571934,0.294731,...,0.500887,0.478557,0.350638,0.214241,0.434134,0.285081,0.496913,0.600883,0.155996,0.457970
ARG,0.279711,0.000000,0.493471,0.309681,0.347328,0.413143,0.658585,0.132035,0.346281,0.401624,...,0.298397,0.342173,0.501926,0.290385,0.442855,0.303057,0.298855,0.417533,0.188903,0.685224
AUS,0.719044,0.493471,0.000000,0.261912,0.614639,0.471694,1.150340,0.519597,0.147340,0.889973,...,0.244101,0.318644,0.993681,0.760207,0.566801,0.763532,0.267807,0.222390,0.646111,1.176979
AUT,0.457924,0.309681,0.261912,0.000000,0.366122,0.387765,0.888528,0.264626,0.168990,0.628161,...,0.197929,0.189730,0.731869,0.503987,0.468920,0.557354,0.111434,0.230485,0.395575,0.915167
BHS,0.374087,0.347328,0.614639,0.366122,0.000000,0.454039,0.541330,0.306204,0.505200,0.438910,...,0.434850,0.410573,0.392880,0.394690,0.286921,0.401578,0.405363,0.493473,0.427946,0.565774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUR,0.285081,0.303057,0.763532,0.557354,0.401578,0.388001,0.386808,0.307087,0.618995,0.327374,...,0.568225,0.585328,0.252418,0.119023,0.439887,0.000000,0.562991,0.685550,0.280536,0.414478
GBR,0.496913,0.298855,0.267807,0.111434,0.405363,0.428954,0.928034,0.298199,0.157531,0.667666,...,0.187667,0.195609,0.771375,0.537264,0.512970,0.562991,0.000000,0.200534,0.423032,0.954672
USA,0.600883,0.417533,0.222390,0.230485,0.493473,0.353160,1.031848,0.400488,0.219127,0.771481,...,0.351793,0.346665,0.875189,0.645605,0.447565,0.685550,0.200534,0.000000,0.531842,1.058486
URY,0.155996,0.188903,0.646111,0.395575,0.427946,0.365350,0.508695,0.167718,0.498825,0.267137,...,0.426110,0.414188,0.396416,0.198982,0.504705,0.280536,0.423032,0.531842,0.000000,0.535333


Now that we have a distance matrix between all alternatives (the agg\_eta).

Perform a **K-Medoid** algorithm using this distance matrix

In [9]:
def K_Medoid_Eta(alternatives, distance_matrix, k=3):
    """ 
    K-Medoid clustering algorithm using the Aggregated Eta matrix
        - alternatives: np.array of the alternatives names only
        - distance_matrix: pd.DataFrame of the distance matrix with index and columns as the alternatives names
        - k: the number of clusters

    Returns:
        - the medoids of the clusters
        - the clusters
    """
    # Initialize medoids
    medoids = np.random.choice(alternatives, k, replace=False) # Randomly select k alternatives
    print("Initial medoids:", medoids)

    # Initialize clusters
    clusters = {medoid: [] for medoid in medoids}

    iter = 0
    # Iterate until convergence
    converged = False
    while not converged:
        iter += 1

        # Assign each alternative to the closest medoid
        for alternative in alternatives:
            distances = [distance_matrix.loc[alternative, medoid] for medoid in medoids]
            closest_medoid = medoids[np.argmin(distances)]
            clusters[closest_medoid].append(alternative)

        # Update medoids
        converged = True
        for medoid in medoids:
            cluster = clusters[medoid]
            distances = [np.sum([distance_matrix.loc[alternative1, alternative2] for alternative1 in cluster]) for alternative2 in cluster] # Sum of distances to all other alternatives in the cluster
            new_medoid = cluster[np.argmin(distances)] # Alternative with the smallest sum of distances
            if new_medoid != medoid: # If the medoid has changed
                medoids[np.where(medoids == medoid)[0][0]] = new_medoid # Replace the medoid
                clusters = {medoid: [] for medoid in medoids} # Reset the clusters
                converged = False # The algorithm has not converged -> continue the iterations
                break # Stop the loop and start a new iteration, this stops the loop:
    
    return medoids, clusters, iter

medoids, clusters, iter = K_Medoid_Eta(alternatives, agg_eta_df, k=5)
print("Converged after", iter, "iterations\nMedoids", medoids ,"\n")


# Print the clusters one by one
for i, cluster in enumerate(clusters):
    print("Cluster with Medoid", medoids[i])
    print(clusters[cluster], "\n")


Initial medoids: ['CHL' 'ISR' 'THA' 'BRN' 'USA']
Converged after 4 iterations
Medoids ['CHL' 'AUT' 'IDN' 'SAU' 'USA'] 

Cluster with Medoid CHL
['ALB', 'ARG', 'BHS', 'BRB', 'BGR', 'CHL', 'CYP', 'HUN', 'MLT', 'PRT', 'ROU', 'LKA', 'URY'] 

Cluster with Medoid AUT
['AUT', 'BEL', 'FIN', 'FRA', 'GRC', 'HKG', 'ISL', 'IRL', 'ISR', 'ITA', 'JPN', 'KOR', 'NLD', 'NZL', 'NOR', 'POL', 'SGP', 'ESP', 'SWE', 'CHE', 'GBR'] 

Cluster with Medoid IDN
['BGD', 'BOL', 'BWA', 'BDI', 'CHN', 'DOM', 'EGY', 'GHA', 'HTI', 'HND', 'IND', 'IDN', 'MUS', 'MEX', 'MAR', 'NPL', 'PAK', 'PRY', 'PHL', 'LCA', 'SDN', 'SYR', 'THA', 'TUR', 'ZMB'] 

Cluster with Medoid SAU
['BHR', 'BRN', 'IRN', 'SAU', 'ZAF', 'TTO'] 

Cluster with Medoid USA
['AUS', 'CAN', 'CZE', 'DNK', 'EST', 'DEU', 'USA'] 



# Try with a smaller dataset


In [11]:
group0 = ["PAK", "SDN", "BDI", "HTI"]
group1 = ["EST", "CZE", "MLT", "SGP", "IRL"]
group2 = ["CHE", "ISL", "NZL", "SWE"]

all_groups = group0 + group1 + group2

# Filter the data to keep only the countries in the groups
data_filtered = data[data["iso3"].isin(all_groups)]
data_filtered = data_filtered.set_index("iso3")
alternatives_filtered = data_filtered.index

phi_c_all_filtered = pf.get_all_Phi_c(data_filtered, P, Q)
eta_filtered = get_eta_matrix(data_filtered, phi_c_all_filtered, W)
agg_eta_filtered = aggregate_all_series(eta_filtered, weight_vector)

agg_eta_filtered_df = pd.DataFrame(agg_eta_filtered, index=alternatives_filtered, columns=alternatives_filtered)


medoids_filtered, clusters_filtered, iter_filtered = K_Medoid_Eta(alternatives_filtered, agg_eta_filtered_df, k=3)

print("Converged after", iter_filtered, "iterations\nMedoids", medoids_filtered ,"\n")

# Print the clusters one by one
for i, cluster in enumerate(clusters_filtered):
    print("Cluster with Medoid", medoids_filtered[i])
    print(clusters_filtered[cluster], "\n")

Initial medoids: ['NZL' 'IRL' 'MLT']
Converged after 5 iterations
Medoids ['ISL' 'CZE' 'SDN'] 

Cluster with Medoid ISL
['ISL', 'NZL', 'SWE', 'CHE'] 

Cluster with Medoid CZE
['CZE', 'EST', 'IRL', 'MLT', 'SGP'] 

Cluster with Medoid SDN
['BDI', 'HTI', 'PAK', 'SDN'] 



Should give back:
- group0 = ["PAK", "SDN", "BDI", "HTI"]
- group1 = ["EST", "CZE", "MLT", "SGP", "IRL"]
- group2 = ["CHE", "ISL", "NZL", "SWE"]

In [ ]:
def G_Kmedoid(data, W, P, Q, Weight_vector, k=3):
    """ 
    Function that receives the raw dataset and creates the clusters using the K-Medoid algorithm using Promethee Gamma as the distance matrix
    - data: pd.DataFrame with the raw dataset - iso3 as index and the criteria as columns names
    - W: list of weights for each criteria - must sum to 1
    - P: list of P thresholds for each criteria - must be > 0
    - Q: list of Q thresholds for each criteria - must be >= 0
    - Weight_vector: list of weights for each time point - must sum to 1
    - k: number of clusters to create
    """
    # Verify that the weights sum to 1
    if sum(W) != 1:
        raise ValueError("The weights must sum to 1")
    
    if len(W) != data.shape[1] or len(P) != data.shape[1] or len(Q) != data.shape[1]:
        raise ValueError("The number of weights, P and Q thresholds must be equal to the number of criteria")
    
    # Verify that the P thresholds are > 0
    if any([p <= 0 for p in P]):
        raise ValueError("The P thresholds must be > 0")
    
    # Verify that the Q thresholds are >= 0
    if any([q < 0 for q in Q]):
        raise ValueError("The Q thresholds must be >= 0")
    
    if sum(Weight_vector) != 1:
        raise ValueError("The time weight vector must sum to 1")
    
    if len(Weight_vector) != data.shape[2]:
        raise ValueError("The time weight vector must have the same length as the time series")
    
    # Get the criteria names
    criterias = data.columns
    alternatives = data.index

    # Computing the distance matrix
    print("Computing the distance matrix using Promethee Gamma...")
    phi_c_all = pf.get_all_Phi_c(data, P, Q)
    eta = get_eta_matrix(data, phi_c_all, W)
    agg_eta = aggregate_all_series(eta, Weight_vector)
    dist_matrix = pd.DataFrame(agg_eta, index=alternatives, columns=alternatives)
    print("Distance matrix computed successfully!\n")

    # Run the K-Medoid algorithm
    print("Running the K-Medoid algorithm...")
    medoids, clusters, iter = K_Medoid_Eta(alternatives, dist_matrix, k)

    print("Converged after", iter, "iterations\nMedoids", medoids ,"\n")

    # Print the clusters one by one
    for i, cluster in enumerate(clusters):
        print("Cluster with Medoid", medoids[i])
        print(clusters[cluster], "\n")

    return medoids, clusters

medoids, clusters = G_Kmedoid(data, W, P, Q, weight_vector, k=3)   